<a href="https://colab.research.google.com/github/Isaac-Jacobson/snow/blob/main/imageFallspeed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Calculate the fall speed from a series of images
Written to find the fallspeed from series of 7 images taken from video of fall speed test rig where 15cm = 780pixels


#Setup


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import os

!mkdir C0001Frames
!mkdir C0004Frames
!mkdir C0007Frames
!mkdir C0008Frames

In [ ]:
!mv /content/*.png C0008Frames

In [ ]:
cap = cv2.VideoCapture("C0001.mp4")
output = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*'MPEG'), 240, (1920, 1080))

In [ ]:
def videoToFrames(videoPath, outputFolder="/content"):
  capture = cv2.VideoCapture(videoPath)
  output = cv2.VideoWriter(outputFolder,cv2.VideoWriter_fourcc(*’MP4’),240,(1920,1080))
  capture = cv2.VideoCapture(videoPath)

  frameNum = 0
  while(True):
    success, frame = capture.read()
    if success:
      path = outputFolder + "//000" + str(frameNum) + ".png"
      cv2.imwrite(path, frame)
    else:
      break
    frameNum += 1
  capture.release()

In [ ]:
videoToFrames("/content/C0001.MP4", "/content/C0001Frames")
videoToFrames("/content/C000.MP4", "/content/C0001Frames")
videoToFrames("/content/C0001.MP4", "/content/C0001Frames")
videoToFrames("/content/C0001.MP4", "/content/C0001Frames")

#Locate the "snowflake"

In [ ]:
def widestRow(arr):
  width = 0
  widestRow = 0
  flag = False
  cnt = 0
  for i in range(0, len(arr)):
    for j in range(0, len(arr[i])):
      if(arr[i][j] >= 1):
        flag = True
        cnt += 1
      else:
        if(flag):
          if(cnt > width):
            widestRow = i
            width = cnt
        flag = False
        cnt = 0
  return widestRow

In [ ]:
def getFlakeY(filename):
  #Load the image
  img = cv2.imread(filename, 0)

  #Get rid of the bottom of the image (lots of glare and we don't need it)
  for i in range(950, len(img)):
    for j in range(0, len(img[i])):
      img[i][j] = 0

  #Get rid of the sides of the image (lots of glare and we don't need it)
  for i in range(0, len(img)):
    for j in range(280, len(img[i])):
      img[i][j] = 0
  for i in range(0, len(img)):
    for j in range(0, 20):
      img[i][j] = 0

  ret,img2 = cv2.threshold(img,25,40,cv2.THRESH_BINARY)
  return widestRow(img2)

#Distance between flake across images

In [ ]:
def getLocations(directory):
  dir_list = os.listdir(directory)
  locations = [0]*len(dir_list)
  for file in dir_list:
    locations[int(file[5])-1] = getFlakeY(directory + "//" + file)
  return locations

def getDistancesP(locations):
  distances = [0]*(len(locations)-1)
  for i in range(0, len(distances)):
    distances[i] = locations[i+1] - locations[i]
  return distances

def getDistancesC(distancesP, pixelSize=0.01923):
  distances = [0.0] * len(distancesP)
  for i in range(0, len(distances)):
    distances[i] = distancesP[i] * pixelSize
  return distances

def getSpeedsMS(distancesC, time=0.10):
  speeds = [0.0] * len(distancesC)
  for i in range(0, len(speeds)):
    speeds[i] = (distancesC[i] / 100) * (1/time)
  return speeds

def average(speeds):
  sum = 0
  for i in speeds:
    sum += i
  return (sum / len(speeds))

#Do the thing

In [ ]:
def details(directory):
  locations = getLocations(directory)
  print("Location of flake (pixels): " + str(locations))
  distancesP = getDistancesP(locations)
  print("Pixel distances between flakes: " + str(distancesP))
  distancesC = getDistancesC(distancesP)
  print("cm distances between flakes: " + str(distancesC))
  speeds = getSpeedsMS(distancesC)
  print("Speeds (m/s): " + str(speeds))
  averageSpeed = average(speeds)
  print("Average speed of snowflake: " + str(averageSpeed) + " m/s")
  speedFt = averageSpeed * 3.28084
  print("Average speed of snowflake: " + str(speedFt) + " ft/s")
  adjustedSpeed = speedFt * (260/60)
  print("Speed adjusted by a factor of (240/60): " + str(adjustedSpeed) + " ft/s")

In [ ]:
details("/content/C0001Frames")

Location of flake (pixels): [77, 164, 247, 349, 436, 526, 607]
Pixel distances between flakes: [87, 83, 102, 87, 90, 81]
cm distances between flakes: [1.67301, 1.59609, 1.96146, 1.67301, 1.7307000000000001, 1.55763]
Speeds (m/s): [0.167301, 0.159609, 0.196146, 0.167301, 0.17307000000000003, 0.155763]
Average speed of snowflake: 0.16986500000000002 m/s
Average speed of snowflake: 0.5572998866000001 ft/s
Speed adjusted by a factor of (240/60): 2.4149661752666667 ft/s


In [ ]:
details("/content/C0004Frames")

Location of flake (pixels): [415, 453, 488, 528, 564, 595, 634]
Pixel distances between flakes: [38, 35, 40, 36, 31, 39]
cm distances between flakes: [0.7307400000000001, 0.67305, 0.7692, 0.69228, 0.59613, 0.74997]
Speeds (m/s): [0.073074, 0.067305, 0.07692, 0.069228, 0.059613, 0.07499700000000001]
Average speed of snowflake: 0.0701895 m/s
Average speed of snowflake: 0.23028051918 ft/s
Speed adjusted by a factor of (240/60): 0.99788224978 ft/s


In [ ]:
details("/content/C0007Frames")

Location of flake (pixels): [422, 484, 547, 611, 690, 756, 820]
Pixel distances between flakes: [62, 63, 64, 79, 66, 64]
cm distances between flakes: [1.19226, 1.21149, 1.23072, 1.5191700000000001, 1.26918, 1.23072]
Speeds (m/s): [0.119226, 0.12114899999999999, 0.12307200000000001, 0.151917, 0.126918, 0.12307200000000001]
Average speed of snowflake: 0.127559 m/s
Average speed of snowflake: 0.41850066956000004 ft/s
Speed adjusted by a factor of (240/60): 1.8135029014266668 ft/s


In [ ]:
details("/content/C0008Frames")

Location of flake (pixels): [52, 150, 249, 348, 455, 560, 669]
Pixel distances between flakes: [98, 99, 99, 107, 105, 109]
cm distances between flakes: [1.88454, 1.90377, 1.90377, 2.05761, 2.0191500000000002, 2.09607]
Speeds (m/s): [0.188454, 0.19037700000000002, 0.19037700000000002, 0.205761, 0.201915, 0.20960700000000002]
Average speed of snowflake: 0.19774850000000002 m/s
Average speed of snowflake: 0.64878118874 ft/s
Speed adjusted by a factor of (240/60): 2.8113851512066663 ft/s
